author: Sam Schmidt<br>
Last successfully run: Apr 26, 2023<br>

In [ ]:
import matplotlib.pyplot as plt
import os
import tables_io
import tempfile
from rail.core.stage import RailStage
from rail.core.data import TableHandle
from rail.core.utils import RAILDIR
from rail.core.utilStages import LSSTFluxToMagConverter, Dereddener

In [ ]:
DS = RailStage.data_store
example_file = os.path.join(RAILDIR, "rail", "examples_data", "testdata", "rubin_dm_dc2_example.pq")
test_data = DS.read_file("test_data", TableHandle, example_file)

In [ ]:
test_data().info()

To convert mags to fluxes, we need to specify patterns for the `flux_name` and `flux_err_name` columns to be converted, and the `mag_name` and `mag_err_name` columsn that will store the newly created magnitudes.  This is done as below, by specifying a string listing the bands, and `{band}` in the patterns where the individual bands will go.  The dictionary below duplicates the default behavior of the converter, but is written out explicitly as an example:

In [ ]:
# convert "gaap" fluxes to magnitudes:
ftomagdict = dict(bands = "ugrizy",
                  flux_name="{band}_gaap1p0Flux",
                  flux_err_name="{band}_gaap1p0FluxErr",
                  mag_name="mag_{band}_lsst",
                  mag_err_name="mag_err_{band}_lsst",
                  copy_cols=dict(ra='ra', decl='decl', objectId='objectId'))
fluxToMag = LSSTFluxToMagConverter.make_stage(name='flux2mag', **ftomagdict)

In [ ]:
mags_data = fluxToMag(test_data)

In [ ]:
list(mags_data().keys())

To deredden magnitudes we need to grab one of the dust maps used by the `dustmaps` package.  We'll grab the default Schlegel-Finkbeiner-Davis "SFD" map.  NOTE: This will download a file to your machine containing the SFD data!

We need to feed the location of the directory containing the newly created "sfd" maps to the stage, as we downloaded the data to the present working directory with the command above, that directory is just `"./"`

In [ ]:
dustmap_dir = "./"

dereddener = Dereddener.make_stage(name='dereddener', dustmap_dir=dustmap_dir)
dereddener.fetch_map()

In [ ]:
deredden_data = dereddener(mags_data)

In [ ]:
deredden_data().keys()

We see that the deredden stage returns us a dictionary with the dereddened magnitudes. Let's plot the difference of the un-dereddened magnitudes and the dereddened ones for u-band to see if they are, indeed, slightly brighter:

In [ ]:
delta_u_mag = mags_data()['mag_u_lsst'] - deredden_data()['mag_u_lsst']
plt.figure(figsize=(8,6))
plt.scatter(mags_data()['mag_u_lsst'], delta_u_mag, s=15)
plt.xlabel("orignal u-band mag", fontsize=12)
plt.ylabel("u - deredden_u");

## for cleanup, uncomment the line below to delete that SFD map directory downloaded in this example


In [ ]:
#! rm -rf sfd/